# RC Snubber Design
[Rudy - Snubber](https://rudys.typepad.com/files/snubber-e-book-complete.pdf)

In [4]:
import numpy as np
from decimal import Decimal


def compute_parasites(f_ring_before, f_ring_after, C_add):
    C = C_add / ((f_ring_before / f_ring_after)**2 - 1)
    L = 1 / (C*(2 * np.pi * f_ring_before)**2)
    parasites = {'C': C,
                 'L': L}

    return parasites


def compute_snubber(parasites, C=None, R=None):
    if C is None:
        C = parasites['C'] * np.array([3, 10])

    if R is None:
        Z = np.sqrt(parasites['L'] / parasites['C'])
        R = Z * np.array([1, 2])

    snubber = {'C': C,
               'R': R}
    return snubber


def compute_snubber_losses(snubber, V, f_sw):
    P_R = snubber['C'] * f_sw * V**2
    return P_R


def compute_peak_voltage(parasites, snubber, Vo, Io):
    Vpk = Vo * (1 + np.sqrt(1 + ((Io / Vo)**2) *
                            (parasites['L'] / parasites['C'])))
    return Vpk


def compute_mosfet_capacitances(Ciss, Coss, Crss):
    capacitances = {'Cgd': Crss,
                    'Cgs': Ciss - Crss,
                    'Cds': Coss - Crss}
    return capacitances

## 0. Parâmetros do Conversor

In [5]:
Vo = 12
Io = 8.5
Po = Io * Vo

print(f'Vo = {Vo} V')
print(f'Io = {Io} A')
print(f'Po = {Po} W')

Vo = 12 V
Io = 8.5 A
Po = 102.0 W


## 1. Snubber para o Diodo:

### 1.1. Inferindo as impedâncias parasitas:

In [8]:
diode_parasites = compute_parasites(f_ring_before=14.55e6,
                                    f_ring_after=10.63e6,
                                    C_add=10e-9)

print(f"Clk = {diode_parasites['C']} F")
print(f"Llk = {diode_parasites['L']} F")

diode_snubber = compute_snubber(diode_parasites)

print(f"Cs range = {diode_snubber['C']} F")
print(f"Rs range = {diode_snubber['R']} Ohms")

Clk = 1.1447871245400463e-08 F
Llk = 1.0451762689710358e-08 F
Cs range = [3.43436137e-08 1.14478712e-07] F
Rs range = [0.95550376 1.91100753] Ohms


### 1.2. Escolhendo uma capacitância:

In [14]:
C = 47e-9
diode_snubber = compute_snubber(diode_parasites,
                                C=C)

print(f"Cs chosen = {diode_snubber['C']} F")
print(f"Rs range = {diode_snubber['R']} Ohms")

Cs chosen = 4.7e-08 F
Rs range = [0.95550376 1.91100753] Ohms


### 1.3. Escolhendo uma resistência:

In [15]:
R = 1
diode_snubber = compute_snubber(diode_parasites,
                                C=diode_snubber['C'],
                                R=R)

print(f"Cs chosen = {diode_snubber['C']} F")
print(f"Rs chosen = {diode_snubber['R']} Ohms")

Cs chosen = 4.7e-08 F
Rs chosen = 1 Ohms


### 1.4. Verificando o pico de tensão com o snubber

In [16]:
Vpk = compute_peak_voltage(diode_parasites,
                           diode_snubber,
                           Vo=Vo,
                           Io=Io)
print(f'Vpk = {Vpk}')

Vpk = 26.49011189453779


### 1.5. Verificando a potência perdida no resistor:

In [17]:
f_sw = 30e3
P_Rs = compute_snubber_losses(diode_snubber,
                              V=Vpk,
                              f_sw=f_sw)
print(f'P_Rs = {P_Rs} W')

diode_snubber_inEff = P_Rs/Po
print(f'inEff = {round(100 * diode_snubber_inEff, 3)} %')

P_Rs = 0.9894336997410368 W
inEff = 0.97 %


## 2. Snubber para o Mosfet:

### 2.1. Inferindo as impedâncias parasitas:

In [56]:
mosfet_parasites = compute_parasites(f_ring_before=2.5e6,
                                     f_ring_after=2.22e6,
                                     C_add=100e-9)

mosfet_snubber = compute_snubber(mosfet_parasites)

print(f"L_lk = {Decimal(str(mosfet_parasites['L'])).to_eng_string()} H")
print(f"C_lk = {Decimal(str(mosfet_parasites['C'])).to_eng_string()} F")
print('---')
print(f"Cs range = {mosfet_snubber['C']} F")
print(f"Rs range = {mosfet_snubber['R']} Ohms")

L_lk = 10.868117547416092E-9 H
C_lk = 372.91162227602885E-9 F
---
Cs range = [1.11873487e-06 3.72911622e-06] F
Rs range = [0.17071599 0.34143198] Ohms


### 2.2. Escolhendo uma capacitância:

In [60]:
mosfet_snubber = compute_snubber(mosfet_parasites,
                                 C=C)

print(f"Cs chosen = {mosfet_snubber['C']} F")
print(f"Rs range = {mosfet_snubber['R']} Ohms")

Cs chosen = 2e-06 F
Rs range = [0.17071599 0.34143198] Ohms


### 2.3. Escolhendo uma resistência:

In [61]:
R = 0.2
mosfet_snubber = compute_snubber(mosfet_parasites, 
                          C=mosfet_snubber['C'],
                          R=R)

print(f"Cs chosen = {mosfet_snubber['C']} F")
print(f"Rs chosen = {mosfet_snubber['R']} Ohms")

Cs chosen = 2e-06 F
Rs chosen = 0.2 Ohms


### 2.4. Verificando o pico de tensão com o snubber

In [62]:
Vpk = compute_peak_voltage(mosfet_parasites,
                           mosfet_snubber,
                           Vo=Vo,
                           Io=Io)
print(f'Vpk = {Vpk}')

Vpk = 125.236024306824


### 2.5. Verificando a potência perdida no resistor:

In [63]:
f_sw =12.5e3
P_Rs = compute_snubber_losses(mosfet_snubber, 
                              V=Vpk, 
                              f_sw=f_sw)
print(f'P_Rs = {P_Rs} W')

mosfet_snubber_inEff = P_Rs/Po
print(f'inEff = {round(100 * mosfet_snubber_inEff, 3)} %')

P_Rs = 392.1015446044853 W
inEff = 4.357 %


In [68]:
# http://rohmfs.rohm.com/en/products/databook/applinote/ic/power/switching_regulator/buck_snubber_app-e.pdf

Rsn = 0.65 * np.sqrt(mosfet_parasites['L'] / mosfet_parasites['C'])
Csn = 1 * mosfet_parasites['C']

print(Rsn)
print(Csn)

0.11096539429726633
3.7291162227602885e-07
